<h3>Read in Kaggle df</h3>

In [ ]:
import pandas as pd
path_kaggle = r'C:\Users\Desktop\MSBA Coursework\Tools for Analytics\datafile'\amazon_com_extras.csv'

with open(path_kaggle):
    #Added fields since line 808 in csv has more than 6 cols and needs to be told to ignore extra data.
    
    fields=['ASIN','GROUP','FORMAT','TITLE','AUTHOR','PUBLISHER']
    df_kaggle = pd.read_csv(path_kaggle, encoding='latin1', dtype=str,usecols=fields)

<h3> Read in amazon_com_extras. </h3>

In [ ]:
import pandas as pd
import random 
import json
from bs4 import BeautifulSoup as bs
import numpy as np
import requests
import time

my_csvfile=r'C:\Users\Desktop\MSBA Coursework\Tools for Analytics\datafile\AZscrape.csv'
start_entry=48001
end_entry=63000 

In [ ]:
Df_read = df_kaggle.iloc[start_entry:end_entry+1]

<h3> Use ISBN 10 to scrape book descriptions</h3>
User Agent Code - Refer https://www.scrapehero.com/how-to-fake-and-rotate-user-agents-using-python-3/

In [ ]:
import requests
import numpy as np

def amazon_description(book_metadata):
    
    import random
    import time
    
    #New columns that will be added to book_metadata
    book_descriptions=list()
    book_price=list()
    book_rating=list()
    
    user_agent_list = [
           #Chrome
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
            #Firefox
            'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
            'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
            'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
            'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
            'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
            'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
        ]
    
    
    book_metadata['Description']=''
    book_metadata['Price']=''
    book_metadata['Rating']=''
    for i in range(len(book_metadata)):
        
        #User Agents to prevent bot detection
        user_agent=random.choice(user_agent_list)
        book_isbn=book_metadata['ASIN'].iloc[i]
        
        print(book_isbn)
        base_url = "https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords="
        url=base_url+book_isbn
        print(url)
        headers={'User-Agent':user_agent}
        book_response=requests.get(url,headers=headers)
        if book_response.status_code != 200:
            print("Getting Book Details Failed")
            return None
        else:
            result_page=bs(book_response.content,'lxml')
            all_book_a_tags=result_page.find("a",class_="a-link-normal s-access-detail-page s-color-twister-title-link a-text-normal")
            try:
                book_url=all_book_a_tags.get("href")
                description,price,rating=get_book_description(book_url,user_agent_list)
            except: 
                continue
            print(description,price,rating)
#             book_descriptions.append(description)
#             book_price.append(price)
#             book_rating.append(rating)
        book_metadata['Description'].iloc[i]=description
        book_metadata['Price'].iloc[i]=price
        book_metadata['Rating'].iloc[i]=rating
        book_metadata.to_csv(my_csvfile,encoding='utf-8') #save data after each query
#     book_metadata['Description']=book_descriptions
#     book_metadata['Price']=book_price
#     book_metadata['Rating']=book_rating
    
    return book_metadata

In [ ]:
def get_book_description(url,user_agent_list):
    book_description=''
    book_rating=''
    book_price=''
    print(url)
    #User Agents to prevent bot detection
    user_agent = random.choice(user_agent_list)
    headers = {'User-Agent': user_agent}
    
    #Throttling to spoof the server into believe these are generated by users
    time.sleep(7)
    
    book_description_response=requests.get(url,headers=headers)
    if book_description_response.status_code!=200:
        print("Error in scraping description")
        return None
    else:
        book_page=bs(book_description_response.content,"lxml")
        a1=book_page.find('div',{'id':'a-page'})
        a2=a1.find('div',class_='book en_US').find('div',class_='a-container').find('div',{'id':'centerCol'})
        try:
            book_description=a2.find('div',{'id':'bookDescription_feature_div'}).find('noscript').find('div').get_text()
        except:
            book_description=np.NAN
        try:
            book_rating=a2.find('div',{'id':'averageCustomerReviews_feature_div'}).find('span',class_='a-declarative').find('span',{'id':'acrPopover'}).find('span',class_='a-declarative').find('a').find('i').find('span').get_text().split()[0]    
        except:
            book_rating=np.NAN
        try:
            book_price=a2.find('div',{'id':'MediaMatrix'}).find('div').find('div',{'id':'tmmSwatches'}).find('ul').find('li',class_='swatchElement selected').find('span').find('span').find('span').find('span',class_='a-color-base').find('span').get_text().strip()
        except:
            book_price=np.NAN
        return book_description,book_price,book_rating

In [ ]:
df_final=amazon_description(Df_read) 

<h2><font color='red'>Note: </font color='red'>Below code not vetted - please do not delete </h2>

<h3> Read in all AZscrape files</h3>

In [ ]:
import pandas as pd
path1 = r'C:\Users\sanjayshah\Desktop\MSBA Coursework\Tools for Analytics\datafile\batch1.csv'
path2 = r'C:\Users\sanjayshah\Desktop\MSBA Coursework\Tools for Analytics\datafile\batch2.csv'
path3 =r'C:\Users\sanjayshah\Desktop\SBA Coursework\Tools for Analytics\datafile\batch3.csv'
path4 = r'C:\Users\sanjayshah\Desktop\MSBA Coursework\Tools for Analytics\datafile\batch4.csv'
path5 = r'C:\Users\sanjayshah\Desktop\MSBA Coursework\Tools for Analytics\datafile\batch5.csv'
path6 = r'C:\Users\sanjayshah\Desktop\MSBA Coursework\Tools for Analytics\datafile\batch6.csv'
path7 = r'C:\Users\sanjayshah\Desktop\MSBA Coursework\Tools for Analytics\datafile\batch7.csv'

with open(path1):
    batch1 = pd.read_csv(path1)
with open(path2):
    batch2 = pd.read_csv(path2)
with open(path3):
    batch3 = pd.read_csv(path3)
with open(path4):
    batch4 = pd.read_csv(path4)
with open(path5):
    batch5 = pd.read_csv(path5)
with open(path6):
    batch6 = pd.read_csv(path6)
with open(path7):
    batch7 = pd.read_csv(path7)

<h3> Combine batches 1-7</h3>

In [ ]:
import os
import pandas as pd

path_dir = r'C:\Users\sanjayshah\MSBA Coursework\Tools for Analytics\datafile'
file_names = ['batch1.csv', 'batch2.csv', 'batch3.csv', 'batch4.csv', 'batch5.csv', 'batch6.csv', 'batch7.csv']
file_name_out = 'df_processed.csv'
path_output = os.path.join(path_dir, file_name_out)
fout = open(path_output, 'w', encoding='utf-8')

for batch in file_names:
    file_input = os.path.join(path_dir,batch)
    fin = open(file_input,'r',encoding='utf-8')
    for line in fin:
        fout.write(line)
fin.close()
fout.close()

<h3> Read in df_processed</h3>

In [ ]:
patha = r'C:\Users\sanjayshah\MSBA Coursework\Tools for Analytics\datafile'
with open(patha):
    df=pd.read_csv(patha)

<h2>Adding Word Cloud Segment</h2>

<h3><font color='red'>Please note</font> : This code has been tested on a subset</h3>

<h3>Remove the following words -</h3>
<li>Short Words</li>
<li>Redundant Words that appear in all descriptions</li>


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS

#MIN_LENGTH and DELETE_WORDS will need to be updated once we identify the concerned words.

MIN_LENGTH=5
DELETE_WORDS=[]

def remove_words(text_string,DELETE_WORDS=DELETE_WORDS):
    for word in DELETE_WORDS:
        text_string = text_string.replace(word,' ')
    return text_string


def remove_short_words(text_string,min_length = MIN_LENGTH):
    word_list = text_string.split()
    for word in word_list:
        if len(word) < min_length:
            text_string = text_string.replace(' '+word+' ',' ',1)
    return text_string


<h3>Categorize data</h3>

<h2>We're trying to group data into the following buckets based on the rating the book has been assigned</h2>

<h3><font color='red'>Group 1</font> : Ratings 5 - 4.5</h3>
<h3><font color='red'>Group 2</font> : Ratings 4.5 - 4</h3>
<h3><font color='red'>Group 3</font> : Ratings 4 - 3.5</h3>
<h3><font color='red'>Group 4</font> : Ratings 3.5 - 3</h3>
<h3><font color='red'>Group 5</font> : Ratings 3 - 2.5</h3>
<h3><font color='red'>Group 0</font> : Irrelevant to the analysis</h3>

In [ ]:
def categorize_data(df):
    
    #This list stores all the categories
    category_list=list()

    
    #Assign categories to each book
    
    for i in range(len(df)):
        if df['Rating'].iloc[i]>=4.5:
            category_list.append(1)
        elif df['Rating'].iloc[i]<4.5 and df['Rating'].iloc[i]>=4:
            category_list.append(2)
        elif df['Rating'].iloc[i]<4 and df['Rating'].iloc[i]>=3.5:
            category_list.append(3)
        elif df['Rating'].iloc[i]<3.5 and df['Rating'].iloc[i]>=3:
            category_list.append(4)
        elif df['Rating'].iloc[i]<3 and df['Rating'].iloc[i]>=2.5:
            category_list.append(5)
        else:
            category_list.append(0)

    #Add categories to the DF
    df['Category']=category_list
    
    return df

<h3><font color='red'>Note: </font color='red'>The following is extra code that is not to be run atm </h3>

<h5>Running the following will convert .ipynb to .py </h5>

In [ ]:
!jupyter nbconvert --to script AZscraping.ipynb

In [ ]:
def get_aiipp(link):
    """Returns lists of Author, ISBN-10, ISBN-13, platform, price, category1, category2, description for AZ best seller list books"""
    import requests
    from bs4 import BeautifulSoup
    page = requests.get(link)
    results_page = BeautifulSoup(page.content,'lxml')
    
    m = results_page.find('a', class_='a-link-normal contributorNameID').get_text()
    book_authors.append(m)
    n = results_page.find('div', class_='content').find_all('li')[5].get_text().replace('ISBN-10:','').strip()
    book_isbn10.append(n)
    p = results_page.find('div', class_='content').find_all('li')[6].get_text().replace('ISBN-13:','').strip()
    book_isbn13.append(p)
    q = int(results_page.find_all('span',class_='a-size-base')[1].get_text().replace('customer reviews','').strip())
    book_num_customer_reviews.append(q)
    s = float(results_page.find_all('span',class_='p13n-sc-price')[1].get_text()[1:])
    book_prices.append(s)
    t = results_page.find('div',class_='a-subheader a-breadcrumb feature').find_all('li')[2].find('a',class_='a-link-normal a-color-tertiary').get_text().replace('\n','').strip()
    book_category1.append(t)
    u = results_page.find('div',class_='a-subheader a-breadcrumb feature').find_all('li')[4].find('a',class_='a-link-normal a-color-tertiary').get_text().replace('\n','').strip()
    book_category2.append(u)
    
    #v = @Akshay to insert description code
    #book_description.append(v)
    
    from time import sleep
    sleep(15)
    

In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep

link_home = 'https://www.amazon.com/best-sellers-books-Amazon/zgbs/books'
page = requests.get(link_home)
results_page = BeautifulSoup(page.content,'lxml')
all_links = results_page.find_all('li',class_='zg-item-immersion')

book_links = list()
book_titles = list()
book_ratings = list()
book_authors = list()
book_isbn10 = list()
book_isbn13 = list()
book_num_customer_reviews = list()
book_prices = list()
book_category1 = list()
book_category2 = list()
book_description = list()

for i in all_links:
    m = 'https://www.amazon.com%s' %(i.find('a', class_='a-link-normal').get('href'))
    book_links.append(m)
    
for i in all_links:
    m = i.find('div',class_='p13n-sc-truncate p13n-sc-line-clamp-1').get_text()
    m = m.replace('\n','')
    m = m.strip()
    book_titles.append(m)

for i in all_links:
    m = i.find_all('a',class_='a-link-normal')[1].get_text()
    if '$' in m:
        book_ratings.append('No Rating')
    else:
        m = m.replace('\n','')
        m = m.strip()
        m = m[:3]
        book_ratings.append(m)
        
        
for i in book_links:
    get_aiipp(i)

    
zip_az = list(zip(book_isbn13,book_isbn10,book_titles,book_authors,book_ratings,book_prices,book_category1,
         book_category2,book_num_customer_reviews,book_links,book_description))
labels = ['ISBN-13','ISBN-10','title','author','rating','price','category1','category2','num_reviews','link','description']
df_az = pd.DataFrame.from_records(zip_az,columns=labels)
df_az.rename(columns={'ISBN-13':'ASIN'})